In [34]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.plotly as plotl
import plotly.graph_objs as go
import plotly.tools as tls
from wordcloud import WordCloud
plt.style.use('ggplot')  

In [105]:
def vocabSavedRange(dl):
	#if dl == 0: return '0'
	if 0 <= dl <= 1: return '0-1'
	elif 1 < dl <= 10: return '2-10'
	elif 10 < dl <= 20: return '11-20'
	elif 20 < dl <= 40: return '21-40'
	elif 40 < dl <= 60: return '41-60'
	elif 60 < dl <= 80: return '61-80'
	elif 80 < dl <= 100: return '81-100'
	elif dl > 100: return '> 100'
	else: return 'None'

def listenScoreRange(dl):
	if dl < 0: return '< 0'
	#elif dl == -1: return 'Incomplete'
	elif 0 <= dl <= 49: return '0-49'
	elif 49 < dl <= 59: return '50-59'
	elif 59 < dl <= 62: return '60-62'
	elif 62 < dl <= 66: return '63-66'
	elif 66 < dl <= 69: return '67-69'
	elif 69 < dl <= 72: return '70-72'
	elif 72 < dl <= 76: return '73-76'
	elif 76 < dl <= 79: return '77-79'
	elif 79 < dl <= 84: return '80-84'
	elif 84 < dl <= 89: return '85-89'
	elif 89 < dl <= 100: return '90-100'
	elif dl > 100: return '>100'
	else: return 'None'


In [23]:
def numOfUsersbyAvgDictionarySize(preppedDataWithoutSection):
	"""
	More than half of students save an avg of 1-20% of words in their dictionaries
	An avg. video vocab count is 239, so between 10 to 25 words is saved from video on average by most users
	"""
	df = pd.read_json(preppedDataWithoutSection) #read_json
	df['avg_words_saved_perc'] = df['avg_words_saved'] * 100
	df = df.groupby(['user']).mean() #avg overall words saved by user, get overall avg of individual averages per video
	df['user_group'] = df['avg_words_saved_perc'].map(vocabSavedRange)
	results = df.groupby(['user_group']).size()
	return results

values = numOfUsersbyAvgDictionarySize('prepped-wo-section.json')
final = go.Bar(
            x=values.values,
            y=values.index,
            orientation = 'h'
)
data = [final]
layout = go.Layout(
    title='Exploring students dictionary usage',
    xaxis=dict(
        title='# of students',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Avg. % of words saved overall',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
plotl.iplot(fig, filename='numOfUsersbyAvgDictionarySize')

In [32]:
def numOfUsersByScoreRange(preppedDataWithoutSection):
	"""
	e.g. users and their score range
	"""
	df = pd.read_json(preppedDataWithoutSection) #read_json in jupyter
	df = df[df['postId'] != 3913]
	df = df.groupby(['user']).mean()
	df['avg_score_range'] = df['avg_score'].map(listenScoreRange)
	results = df.groupby(['avg_score_range']).size()
	return results #df[['avg_score', 'avg_score_range']]

values2 = numOfUsersByScoreRange('prepped-wo-section.json')
final = go.Bar(
            x=values2.values,
            y=values2.index,
            orientation = 'h'
)
data = [final]
layout = go.Layout(
    title='Exploring overall students performance',
    xaxis=dict(
        title='# of students',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Avg. score',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
plotl.iplot(fig, filename='numOfUsersByScoreRange')

In [113]:
def AvgScorebyAvgDictionarySize(preppedDataWithoutSection):
	df = pd.read_json(preppedDataWithoutSection) #read_json
	filtered = df[df['avg_score'] >= 0] # ignore Nan, means no scores for that user
	format = filtered.groupby(['user']).mean()
	format['avg_words_saved'] = (100 * format['avg_words_saved']).map(vocabSavedRange) #convert to percent and map to a range
	final = format.groupby(['avg_words_saved'])['avg_score'].mean()
	return final

values3 = AvgScorebyAvgDictionarySize('prepped-wo-section.json')
final = go.Bar(
            x=values3.values,
            y=values3.index,
            orientation = 'h'
)
data = [final]
layout = go.Layout(
    title='Explore relationship between scores and dictionary usage',
    xaxis=dict(
        title='Avg. listen score',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Avg. % of words saved overall',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
plotl.iplot(fig, filename='AvgScorebyAvgDictionarySize')

In [137]:
#http://stackoverflow.com/questions/17812978/how-to-plot-two-columns-of-a-pandas-data-frame-using-points
#http://stackoverflow.com/questions/28576540/how-can-i-normalize-the-data-in-a-range-of-columns-in-my-pandas-dataframe

##############################################
#     REMOVE STOP WORDS WITH NLTK
##############################################

import nltk
import json
from nltk import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

video = pd.read_json('videoDataInfoNew.json')
student_behav = pd.read_json('dictionary.json')
stop = stopwords.words('english')

video['stop_word_filtered'] = video['wordList'].apply(lambda x: [item for item in x if item not in stop])
#video[['postId','wordList','stop_word_filtered']]
#video.to_json('video_data_stop_words_removed.json', orient='records')

student_behav['stop_word_filtered'] = student_behav['wordList'].apply(lambda x: [item for item in x if item not in stop])
student_behav[['memberId','wordList','stop_word_filtered']]
student_behav.to_json('student_dictionary_stop_words_removed.json', orient='records')

allWords = []
def allWordss(wlist):
    for word in wlist:
        allWords.append(word)
stop
#student_behav['wordList'].apply(allWordss)
#freq = Counter(allWords)
#freq.most_common(200)
#freq.to_json('freq_dict_words_among_users.json', orient='records')

#wordcloud = WordCloud().generate(' '.join(allWords))
#plt.imshow(wordcloud)
#plt.axis("off")
#plt.show()


[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u'her',
 u'hers',
 u'herself',
 u'it',
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'against',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'then',
 u'once',
 u'here',
 u'there',
 u'when',
 u'where',
 u'why',
 u'how',
 u'all

In [131]:
##########################################################
#https://dev.socrata.com/blog/2016/02/02/plotly-pandas.html
# Correlation between words saved and score
#########################################################

df = pd.read_json('prepped-wo-section.json') #read_json
df['avg_words_saved_perc'] = df['avg_words_saved'] * 100
df = df.groupby(['user']).mean() #avg overall words saved by user, get overall avg of individual averages per video
df['user_group'] = df['avg_words_saved_perc'].map(vocabSavedRange)

data = pd.DataFrame({'score':df['avg_score'], 'words_saved':df['avg_words_saved_perc'] })

color1 = '#9467bd'
color2 = '#F08B00'

trace1 = go.Scatter(
    x = data.index,
    y = data['score'],
    name='scores',
    line = dict(
        color = color1
    )
)
trace2 = go.Scatter(
    x= data.index,
    y =data['words_saved'] ,
    name='words_saved',
    yaxis='y2',
    mode='markers'

)
data = [trace1, trace2]
layout = go.Layout(
    title= "Relationship between scores and words saved",
    yaxis=dict(
        title='score',
        titlefont=dict(
            color=color1
        ),
        tickfont=dict(
            color=color1
        )
    ),
    yaxis2=dict(
        title='words',
        overlaying='y',
        side='right',
        titlefont=dict(
            color=color2
        ),
        tickfont=dict(
            color=color2
        )

    )

)
fig = go.Figure(data=data, layout=layout)
plotl.iplot(fig)

In [135]:
data = pd.DataFrame({'score':df['avg_score'], 'words_saved':df['avg_words_saved_perc'] })

data = [
    go.Scatter(
        x=data['score'], # assign x as the dataframe column 'x'
        y=data['words_saved']
    )
]
#plotl.iplot(data)
#fig = go.Figure(data=data, layout=layout)
#plotl.iplot(fig)